In [ ]:
---
comments: true
layout: post
title: CSSE 2 Final Week 21
author: Lucas Masterson
description: Personal blog for CSSE 2 Final Week 21
permalink: /csse/final/week20
type: ccc
---

## Day 1:

Tried integrating the adventure game with the platformer game. Was wholly unsuccessful, but I'll try another approach. My origianl approach was loading the adventure game files and platformer files into the same repo, the connecting them using imports + integration file.

Here's my integration file: `PlatformerIntegration.js`:

```js
// File: _site/assets/js/adventureGame/PlatformerIntegration.js
import GameControl from '../platformer/GameControl.js';
import GameEnv from '../platformer/GameEnv.js';

export function transitionToPlatformerLevel(levelTag) {
    // Find the platformer game level by tag
    //const level = GameEnv.levels.find(l => l.tag === levelTag);
    console.log("transiitoning to level", levelTag);
    if (level) {
        // Transition to the platformer game level
        console.log(`Transitioned to level with tag ${levelTag}.`);
        GameControl.transitionToLevel(levelTag);
    } else {
        console.error(`Level with tag ${levelTag} not found.`);
    }
}
```

and here's `adventureGame.md`:

```html
<script type="module">
    import AdventureGameControl from '{{site.baseurl}}/assets/js/adventureGame/GameControl.js';
    import Prompt from '{{site.baseurl}}/assets/js/adventureGame/Prompt.js';
    import { getStats } from '{{site.baseurl}}/assets/js/adventureGame/StatsManager.js';
    import GameSetup from '{{site.baseurl}}/assets/js/platformer/GameSetup.js';
    import PlatformerGameControl from '{{site.baseurl}}/assets/js/platformer/GameControl.js';
    import SettingsControl from '{{site.baseurl}}/assets/js/platformer/SettingsControl.js';
    import GameEnv from '{{site.baseurl}}/assets/js/platformer/GameEnv.js';
    import Leaderboard from '{{site.baseurl}}/assets/js/platformer/Leaderboard.js';
```

which amounted to 37 changed files of 1860 additions. 

This looks to not be working, but I think I'll try writing the platformer game partially from scratch so I can better start envs inside adventure. Painnnnnn.


## Day 2:

Tried some integration again but was futile. For a bit tried to build the platformer game from scratch in the adventure game environment, but the attempt was futile (didn't really know where to start).

Following the rather dissapointing attempt, I had inspiration to actually build a boss fight level with actual fighting mechanics. I've made a few additions:

I built this file to operate bossfights called `BossFight.js`:

```js
import GameEnv from './GameEnv.js';
import GameControl from './GameControl.js';
import Enemy from './Enemy.js';
import hpBar from './hpBar.js';
export class Boss extends Enemy {
    // instantiation: constructor sets up player object
    constructor(canvas, image, data, xPercentage, yPercentage, name, minPosition) {
        super(canvas, image, data, xPercentage, yPercentage, name, minPosition);
        this.storeSpeed = this.speed;
        this.animationSpeed = data?.animationSpeed || 1; //higher "animationSpeed" means slower animation
        this.counter = data?.animationSpeed;
        this.enemySpeed();
        //Hp Bar
        this.maxHp = 100; // Maximum health points
        this.currentHp = 100; // Current health points
        this.hpBar = new hpBar(100, 15, this.canvasWidth, this.canvasHeight, this.maxHp, this.currentHp, this.x, this.y)
        this.attackRange = 50;
        this.laserCooldown = 100; // Cooldown period for firing lasers
        this.laserCounter = 0;
    }
    //overwrite the method
    updateFrameX() {
        // Update animation frameX of the object
        if(!this.state.isDying || this.state.animation != "death"){
            if (this.frameX < this.maxFrame) {
                if(this.counter > 0){
                    this.frameX = this.frameX;
                    this.counter--;
                }
                else{
                    this.frameX++
                    this.counter = this.animationSpeed;
                }
            } else {
                this.frameX = this.minFrame;
            }
        }
        else if(this.state.isDying && this.state.animation == "death"){
            this.animationSpeed = 50;
            if (this.frameX < this.maxFrame) {
                if(this.counter > 0){
                    this.frameX = this.frameX;
                    this.counter--;
                }
                else{
                    this.frameX++
                    this.counter = this.animationSpeed;
                }
            } else {
                this.destroy();
                this.hpBar.destroy();
            }
        }
    }
    //overwrite the method
    updateMovement(){
        if (this.state.animation === "right") {
            this.speed = Math.abs(this.storeSpeed)
        }
        else if (this.state.animation === "left") {
            this.speed = -Math.abs(this.storeSpeed);
        }
        else if (this.state.animation === "death") {
            this.speed = 0
        }
        else if (this.state.animation === "idleL") {
            this.speed = 0
        }
        else if (this.state.animation === "idleR") {
            this.speed = 0
        }
        // Move the enemy
        this.x += this.speed;
        this.playerBottomCollision = false;
    }
    fireLaser() {
        if (this.laserCounter <= 0) {
            // Logic to fire laser beams downwards
            const laser = {
                x: this.x,
                y: this.y + this.canvasHeight,
                width: 5,
                height: 20,
                speed: 5,
                update: function() {
                    this.y += this.speed;
                },
                draw: function(ctx) {
                    ctx.fillStyle = 'red';
                    ctx.fillRect(this.x, this.y, this.width, this.height);
                }
            };
            this.lasers = this.lasers || [];
            this.lasers.push(laser);
            this.laserCounter = this.laserCooldown;
        } else {
            this.laserCounter--;
        }
    }
    update() {
        super.update();
        this.updateMovement();
        this.updateFrameX();
        this.fireLaser();
        this.hpBar.updateHpBar(this.currentHp, this.x, this.y, this.canvasWidth, this.canvasHeight);
    }
    //overwrite the method
    collisionAction() {
        if (this.collisionData.touchPoints.other.id === "player") {
            if (this.collisionData.touchPoints.other.right && !this.collisionData.touchPoints.other.bottom) {
                this.x -= 10;
                this.state.direction = "left";
                this.state.animation = "attackL";
                this.speed = 0;
            }
            else if(this.collisionData.touchPoints.other.left && !this.collisionData.touchPoints.other.bottom){
                this.x += 10;
                this.state.direction = "right";
                this.state.animation = "attackR";
                this.speed = 0;
            }
            else if(this.collisionData.touchPoints.other.bottom && this.immune == 0){
                GameEnv.goombaBounce = true;
            }
        }
        else{
            if(this.currentHp < 0){
                this.state.animation = "death";
                if(!this.state.isDying && this.state.animation == "death"){
                    this.frameX = 0;
                }
                this.state.isDying = true;
                GameEnv.invincible = true;
                GameEnv.playSound("goombaDeath");
            }
            else{
                if (GameEnv.playerAttack && (Math.abs((this.x + this.canvasWidth)/2-(GameEnv.x + GameEnv.canvasWidth)/2) < (this.canvasWidth/2 + this.attackRange))) {
                    this.currentHp -= 1;
                }
            }
        }
    }
}
export default Boss;
```

and made a few changes to `GameSetterBoss.js` to change the sprites. The level seems to fail, however, when I remove the boss enemy from the `enemies` object of the level assets or modify it with anything else. I also happened to notice quite a few errors with `key is undefined`, which is a core mechanic...

```console
Uncaught ReferenceError: keys is not defined
    updateParallaxDirection https://nighthawkcoders.github.io/platformer4x/assets/js/platformer/GameEnv.js:240
    handleKeyDown https://nighthawkcoders.github.io/platformer4x/assets/js/platformer/PlayerBase.js:135
    PlayerBase https://nighthawkcoders.github.io/platformer4x/assets/js/platformer/PlayerBase.js:38
    PlayerHills https://nighthawkcoders.github.io/platformer4x/assets/js/platformer/PlayerHills.js:24
    load https://nighthawkcoders.github.io/platformer4x/assets/js/platformer/GameLevel.js:56
    transitionToLevel https://nighthawkcoders.github.io/platformer4x/assets/js/platformer/GameControl.js:229
    gameLoop https://nighthawkcoders.github.io/platformer4x/assets/js/platformer/GameControl.js:269
    gameLoop https://nighthawkcoders.github.io/platformer4x/assets/js/platformer/GameControl.js:280
    gameLoop https://nighthawkcoders.github.io/platformer4x/assets/js/platformer/GameControl.js:280
    <anonymous> https://nighthawkcoders.github.io/platformer4x/:239
GameEnv.js:240:21
```